In [27]:
import pandas as pd
import numpy as np
import yfinance as yf #yahoo finance

In [28]:
symbol = "AAPL"  # Replace with the stock symbol you need

# Get 5-minute data (closest to 10-minute)
df_5min = yf.download(symbol, interval="5m", period="60d")

# Get 1-hour data (closest to 4-hour)
df_1hour = yf.download(symbol, interval="1h", period="730d")

# Flatten column names (remove multi-level index)
df_5min.columns = [col[0] for col in df_5min.columns]
df_1hour.columns = [col[0] for col in df_1hour.columns]

# Resample 5-minute data to 10-minute
df_10min = df_5min.resample('10T').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).dropna()

# Resample 1-hour data to 4-hour
df_4hour = df_1hour.resample('4H').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).dropna()

print("10-Minute Data:\n", df_10min.head())
print("4-Hour Data:\n", df_4hour.head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

10-Minute Data:
                                  Open        High         Low       Close  \
Datetime                                                                    
2024-10-30 13:30:00+00:00  232.619995  233.229904  231.850006  232.759995   
2024-10-30 13:40:00+00:00  232.770004  232.779999  232.009995  232.059998   
2024-10-30 13:50:00+00:00  232.119995  232.679993  232.000000  232.589996   
2024-10-30 14:00:00+00:00  232.598999  232.781998  232.059998  232.080002   
2024-10-30 14:10:00+00:00  232.074997  232.309906  232.039993  232.279999   

                            Volume  
Datetime                            
2024-10-30 13:30:00+00:00  3862281  
2024-10-30 13:40:00+00:00  1078355  
2024-10-30 13:50:00+00:00   973273  
2024-10-30 14:00:00+00:00   861328  
2024-10-30 14:10:00+00:00   675845  
4-Hour Data:
                                  Open        High         Low       Close  \
Datetime                                                                    
2022-03-02 12:00


/var/folders/g9/01xw9sf55fbg7wx6rnn2dsc40000gn/T/ipykernel_3210/553440689.py:14: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_10min = df_5min.resample('10T').agg({
/var/folders/g9/01xw9sf55fbg7wx6rnn2dsc40000gn/T/ipykernel_3210/553440689.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4hour = df_1hour.resample('4H').agg({


In [29]:
df_10min.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2304 entries, 2024-10-30 13:30:00+00:00 to 2025-01-28 20:50:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    2304 non-null   float64
 1   High    2304 non-null   float64
 2   Low     2304 non-null   float64
 3   Close   2304 non-null   float64
 4   Volume  2304 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 108.0 KB


In [30]:
def identify_fractals(df):
    df['Bullish_Fractal'] = (df['Low'] < df['Low'].shift(1)) & (df['Low'] < df['Low'].shift(2)) & \
                            (df['Low'] < df['Low'].shift(-1)) & (df['Low'] < df['Low'].shift(-2))

    df['Bearish_Fractal'] = (df['High'] > df['High'].shift(1)) & (df['High'] > df['High'].shift(2)) & \
                            (df['High'] > df['High'].shift(-1)) & (df['High'] > df['High'].shift(-2))
    return df

# Identify fractals in 10-minute data
df_10min = identify_fractals(df_10min)

# Display detected fractals
fractals = df_10min[(df_10min['Bullish_Fractal']) | (df_10min['Bearish_Fractal'])]
print(fractals[['High', 'Low', 'Bullish_Fractal', 'Bearish_Fractal']])

                                 High         Low  Bullish_Fractal  \
Datetime                                                             
2024-10-30 14:00:00+00:00  232.781998  232.059998            False   
2024-10-30 15:10:00+00:00  231.054993  230.505005             True   
2024-10-30 15:30:00+00:00  231.664993  231.089996            False   
2024-10-30 16:00:00+00:00  231.549896  231.310104            False   
2024-10-30 16:40:00+00:00  231.389999  231.210007            False   
...                               ...         ...              ...   
2025-01-28 18:40:00+00:00  239.059998  238.610001            False   
2025-01-28 19:00:00+00:00  238.951202  238.589996             True   
2025-01-28 19:40:00+00:00  239.479996  239.162506            False   
2025-01-28 20:10:00+00:00  238.979996  238.309998             True   
2025-01-28 20:20:00+00:00  239.250000  238.789993            False   

                           Bearish_Fractal  
Datetime                                    

In [31]:
fractals

,Open,High,Low,Close,Volume,Bullish_Fractal,Bearish_Fractal
Datetime,,,,,,,
2024-10-30 14:00:00+00:00,232.598999,232.781998,232.059998,232.080002,861328,False,True
2024-10-30 15:10:00+00:00,231.050003,231.054993,230.505005,230.919998,1273217,True,False
2024-10-30 15:30:00+00:00,231.559998,231.664993,231.089996,231.119995,864861,False,True
2024-10-30 16:00:00+00:00,231.384995,231.549896,231.310104,231.354996,426960,False,True
2024-10-30 16:40:00+00:00,231.384995,231.389999,231.210007,231.289993,284730,False,True
...,...,...,...,...,...,...,...
2025-01-28 18:40:00+00:00,238.714996,239.059998,238.610001,238.919296,614361,False,True
2025-01-28 19:00:00+00:00,238.809998,238.951202,238.589996,238.684998,547250,True,False
2025-01-28 19:40:00+00:00,239.190002,239.479996,239.162506,239.410004,690493,False,True


In [43]:
# Identify fractals in both datasets
df_10min = identify_fractals(df_10min)
df_4hour = identify_fractals(df_4hour)

# Match fractals from 10-minute to 4-hour data
df_10min['Fractal_Timestamp'] = df_10min.index
df_4hour['Fractal_Timestamp'] = df_4hour.index

matched_fractals = pd.merge_asof(
    df_10min[df_10min['Bullish_Fractal'] | df_10min['Bearish_Fractal']],
    df_4hour[df_4hour['Bullish_Fractal'] | df_4hour['Bearish_Fractal']],
    on='Fractal_Timestamp',
    direction='backward'
)

print("Matched Fractals Between 10-Minute and 4-Hour Data:")
matched_fractals[['Fractal_Timestamp', 'Bullish_Fractal_x', 'Bearish_Fractal_x', 'Bullish_Fractal_y', 'Bearish_Fractal_y']]

Matched Fractals Between 10-Minute and 4-Hour Data:


,Fractal_Timestamp,Bullish_Fractal_x,Bearish_Fractal_x,Bullish_Fractal_y,Bearish_Fractal_y
0,2024-10-30 14:00:00+00:00,False,True,False,True
1,2024-10-30 15:10:00+00:00,True,False,False,True
2,2024-10-30 15:30:00+00:00,False,True,False,True
3,2024-10-30 16:00:00+00:00,False,True,False,True
4,2024-10-30 16:40:00+00:00,False,True,False,True
...,...,...,...,...,...
573,2025-01-28 18:40:00+00:00,False,True,True,False
574,2025-01-28 19:00:00+00:00,True,False,True,False
575,2025-01-28 19:40:00+00:00,False,True,True,False
576,2025-01-28 20:10:00+00:00,True,False,True,False


In [45]:
# Define Buy/Sell/Hold conditions
def classify_trade(row):
    if row['Bullish_Fractal_x'] and row['Bullish_Fractal_y']:
        return "BUY"
    elif row['Bearish_Fractal_x'] and row['Bearish_Fractal_y']:
        return "SELL"
    else:
        return "HOLD"

# Apply function to create the 'Buy_or_Sell' column
matched_fractals['Buy_or_Sell'] = matched_fractals.apply(classify_trade, axis=1)

# Display the updated DataFrame
print(matched_fractals[['Fractal_Timestamp', 'Bullish_Fractal_x', 'Bearish_Fractal_x', 
                        'Bullish_Fractal_y', 'Bearish_Fractal_y', 'Buy_or_Sell']])


            Fractal_Timestamp  Bullish_Fractal_x  Bearish_Fractal_x  \
0   2024-10-30 14:00:00+00:00              False               True   
1   2024-10-30 15:10:00+00:00               True              False   
2   2024-10-30 15:30:00+00:00              False               True   
3   2024-10-30 16:00:00+00:00              False               True   
4   2024-10-30 16:40:00+00:00              False               True   
..                        ...                ...                ...   
573 2025-01-28 18:40:00+00:00              False               True   
574 2025-01-28 19:00:00+00:00               True              False   
575 2025-01-28 19:40:00+00:00              False               True   
576 2025-01-28 20:10:00+00:00               True              False   
577 2025-01-28 20:20:00+00:00              False               True   

     Bullish_Fractal_y  Bearish_Fractal_y Buy_or_Sell  
0                False               True        SELL  
1                False             